In [1]:
# Se instalan los paquetes correspondientes, la mayoria de ellos ya se encontraban en el enviroment
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import zipfile
import shutil
import os
import time
import glob


In [2]:
#INICIALIZANDO EL DRIVER CON LA PAGINA WEB
# https://www.bumeran.com.pe/empleos.html


driver = webdriver.Chrome()
driver.maximize_window()# sin ninguna acción maximiza
url = 'https://proyectos.inei.gob.pe/microdatos/'
driver.get(url)


In [3]:
#INGRESO A LA WEB DEL INEI MICRODATOS EXACTAMENTE AL APARTADO ENCUESTAS
ingreso = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="jsmenu"]/li[1]/a'))
)

ingreso.click()

#ENCUESTA Y METODOLOGIA
tipo_encuesta = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="Formulario"]/form[1]/div[4]/div/span/span[1]/span/span[2]/b'))
)

tipo_encuesta.click()


area_box = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/span/span/span[1]/input'))
)

# porsia se limpia el cuadro PARA EVITAR CONFUSIONES
area_box.clear()
area_box.send_keys("ENAHO Metodología ACTUALIZADA") # DE QUERERE CAMBIAR EL TIPO DE METODOLOGIA HACERLO ACA
area_box.send_keys(Keys.RETURN)
time.sleep(3) # espero porque sino no se selecciona

# TIPO DE ENCUESTA, PANEL O NO 

tipo_2= WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="ENAHON"]/select')))
tipo_2.click()
tipo_2_1 = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ENAHON"]/select')))
select_tipo = Select(tipo_2_1)
select_tipo.select_by_visible_text("Condiciones de Vida y Pobreza - ENAHO") #DE CAMBIAR EL TIPO DE ENCUESTA SOLO CAMBIAR EL NOMBRE ACA
# ahora hago click afuera para salir de ese cuadro 
ActionChains(driver).move_by_offset(0, 0).click().perform() 
# Esperar 20 segundos si lo necesitas
time.sleep(2)


###############################################




In [4]:
# Configura carpeta de descargas temporal
download_dir = r"C:\Users\FERNANDO\Downloads"
output_dir = r"C:\Users\FERNANDO\Documents\PI_INEQUIDAD\scripts_data\data\inputs\sumaria_enaho"

os.makedirs(download_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

for year in range(2007, 2024):

    # SELECCIONO EL AÑO
    año = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="divAnio"]/select')))
    select_año = Select(año)

    # SELECCIONO EL AÑO DESEADO, COMO EJEMPLO 2007
    select_año.select_by_visible_text(f"{year}")
    time.sleep(2)

    # SELECCIONO TIPO anual, trimestral, etc
    # Espera a que el elemento esté presente
    periodicidad=WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="divPeriodo"]')))
    periodicidad.click()
    selec_periodicidad=  WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="divPeriodo"]/select')))
    select_per= Select(selec_periodicidad)
    select_per.select_by_visible_text("Anual - (Ene-Dic)")
    ActionChains(driver).move_by_offset(0, 0).click().perform() 

    #DESCARGAR STATA ARCHIVO
    link_descarga = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="divDetalle"]/table/tbody/tr/td/table/tbody/tr[25]/td[10]/a'))
    )

    # Haz clic para iniciar la descarga
    link_descarga.click()
    time.sleep(2)
